In [ ]:
# Black-Scholes Greeks (analytic) and Monte Carlo estimators (finite-difference + pathwise for Delta)
# Executable example: computes analytic Greeks and Monte Carlo estimates side-by-side.
import numpy as np
from math import erf, sqrt, exp, pi
np.random.seed(42)

# --- Normal helpers (no scipy dependency) ---
def norm_cdf(x):
    return 0.5 * (1.0 + erf(x / sqrt(2.0)))

def norm_pdf(x):
    return np.exp(-0.5 * x * x) / np.sqrt(2.0 * np.pi)

# --- Black-Scholes price & Greeks (European call) ---
def bs_price_call(S, K, r, sigma, T):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0.0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm_cdf(d1) - K * np.exp(-r * T) * norm_cdf(d2)

def bs_greeks_call(S, K, r, sigma, T):
    if T <= 0 or sigma <= 0:
        return {"delta": 1.0 if S>K else 0.0, "gamma": 0.0, "vega": 0.0, "theta": 0.0, "rho": 0.0}
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    pdf_d1 = norm_pdf(d1)
    delta = norm_cdf(d1)
    gamma = pdf_d1 / (S * sigma * np.sqrt(T))
    vega = S * pdf_d1 * np.sqrt(T)           # note: per 1.0 vol (not per 1%)
    theta = - (S * pdf_d1 * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r*T) * norm_cdf(d2)
    rho = K * T * np.exp(-r*T) * norm_cdf(d2)
    return {"delta": delta, "gamma": gamma, "vega": vega, "theta": theta, "rho": rho}

# --- Monte Carlo pricing (direct) and finite-difference Greeks with Common Random Numbers ---
def mc_price_call(S0, K, r, sigma, T, n_paths=100000, antithetic=True):
    dt = T
    n = n_paths
    z = np.random.normal(size=n//1)
    if antithetic:
        z = np.concatenate([z, -z])
    ST = S0 * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    payoffs = np.maximum(ST - K, 0.0)
    price = np.exp(-r * T) * np.mean(payoffs)
    return price, z, payoffs, ST

def mc_greeks_finite_diff(S0, K, r, sigma, T, n_paths=200000, h_S=0.1, h_sigma=0.01, h_T=1/365.0, antithetic=True):
    # Use common random numbers for all bumps (variance reduction)
    n = n_paths
    z = np.random.normal(size=n//1)
    if antithetic:
        z = np.concatenate([z, -z])
    sqrtT = np.sqrt(T)
    # helper to compute ST for given S0 and sigma with same z
    def ST_for(S0_local, sigma_local, T_local):
        return S0_local * np.exp((r - 0.5 * sigma_local**2) * T_local + sigma_local * np.sqrt(T_local) * z)
    # base price
    ST = ST_for(S0, sigma, T)
    payoffs = np.maximum(ST - K, 0.0)
    P = np.exp(-r * T) * np.mean(payoffs)
    # Delta (finite difference)
    ST_plus = ST_for(S0 + h_S, sigma, T)
    ST_minus = ST_for(S0 - h_S, sigma, T)
    Pay_plus = np.maximum(ST_plus - K, 0.0)
    Pay_minus = np.maximum(ST_minus - K, 0.0)
    delta_fd = np.exp(-r * T) * np.mean((Pay_plus - Pay_minus) / (2.0 * h_S))
    # Delta (pathwise estimator) - unbiased for European call (where payoff is differentiable almost everywhere)
    # pathwise derivative: d/dS0 [exp(-rT) * max(ST-K,0)] = exp(-rT) * I(ST>K) * (ST/S0)
    pathwise = np.exp(-r*T) * (ST > K) * (ST / S0)
    delta_pathwise = np.mean(pathwise)
    # Gamma (finite diff)
    gamma_fd = np.exp(-r*T) * np.mean((Pay_plus - 2*payoffs + Pay_minus) / (h_S**2))
    # Vega (finite diff)
    ST_vegaplus = ST_for(S0, sigma + h_sigma, T)
    ST_vegaminus = ST_for(S0, sigma - h_sigma, T)
    pay_vegaplus = np.maximum(ST_vegaplus - K, 0.0)
    pay_vegaminus = np.maximum(ST_vegaminus - K, 0.0)
    vega_fd = np.exp(-r*T) * np.mean((pay_vegaplus - pay_vegaminus) / (2.0 * h_sigma))
    # Theta (finite diff) - bump time forward by h_T (T must stay >0)
    T_minus = max(1e-8, T - h_T)
    ST_time = ST_for(S0, sigma, T_minus)  # note: we use same z but different sqrt(T)
    pay_time = np.maximum(ST_time - K, 0.0)
    theta_fd = (np.exp(-r*T_minus) * np.mean(pay_time) - np.exp(-r*T) * np.mean(payoffs)) / (-h_T)  # per year
    # Rho (finite diff) - bump interest rate
    h_r = 1e-4
    # For rho, reroll ST using bumped r but same z (affects drift)
    def ST_for_r(r_local):
        return S0 * np.exp((r_local - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_r_plus = ST_for_r(r + h_r)
    pay_r_plus = np.maximum(ST_r_plus - K, 0.0)
    rho_fd = (np.exp(-(r + h_r)*T) * np.mean(pay_r_plus) - np.exp(-r*T) * np.mean(payoffs)) / h_r
    results = {
        "price": P,
        "delta_fd": delta_fd,
        "delta_pathwise": delta_pathwise,
        "gamma_fd": gamma_fd,
        "vega_fd": vega_fd,
        "theta_fd": theta_fd,
        "rho_fd": rho_fd,
        "n_paths": len(z)
    }
    return results

# --- Example run ---
S0 = 100.0
K = 100.0
r = 0.01
sigma = 0.2
T = 30/365.0  # 30 days to expiry (in years)
n_mc = 200_000

print("Parameters: S0={}, K={}, r={}, sigma={}, T={} years, MC paths={}".format(S0, K, r, sigma, T, n_mc))
bs_price = bs_price_call(S0, K, r, sigma, T)
bs_greeks = bs_greeks_call(S0, K, r, sigma, T)
print("\nBlack-Scholes (analytic) -> Price: {:.6f}".format(bs_price))
for k,v in bs_greeks.items():
    print(f"  {k:7s}: {v:.6f}")

mc_results = mc_greeks_finite_diff(S0, K, r, sigma, T, n_paths=n_mc, h_S=0.1, h_sigma=0.01, h_T=1/365.0, antithetic=True)
print("\nMonte Carlo (CRN finite-diff + pathwise for Delta):")
print("  Price (MC)       : {:.6f}".format(mc_results["price"]))
print("  Delta (FD)       : {:.6f}".format(mc_results["delta_fd"]))
print("  Delta (pathwise) : {:.6f}".format(mc_results["delta_pathwise"]))
print("  Gamma (FD)       : {:.6f}".format(mc_results["gamma_fd"]))
print("  Vega (FD)        : {:.6f}".format(mc_results["vega_fd"]))
print("  Theta (FD)       : {:.6f}".format(mc_results["theta_fd"]))
print("  Rho (FD)         : {:.6f}".format(mc_results["rho_fd"]))

# Print differences
print("\nDifferences (MC - BS analytic):")
print("  Price diff : {:.6e}".format(mc_results["price"] - bs_price))
print("  Delta diff : {:.6e}".format(mc_results["delta_pathwise"] - bs_greeks["delta"]))
print("  Gamma diff : {:.6e}".format(mc_results["gamma_fd"] - bs_greeks["gamma"]))
print("  Vega diff  : {:.6e}".format(mc_results["vega_fd"] - bs_greeks["vega"]))
